In [23]:
import anndata
import cv2
import io
import matplotlib.pyplot as plt
import numpy as np
import openslide
import os
import pandas as pd
import pickle
import scanpy as sc
import shutil
import tarfile
import tifffile
from mcDETECT.utils import *

import warnings
warnings.filterwarnings("ignore")
sc.settings.verbosity = 0

In [48]:
# Read gene panel
utils_dir = "../../data/_utils/"
gene_panel = np.load(utils_dir + "shared_genes.npy").astype(str).tolist()
gene_panel_set = set(gene_panel)

In [49]:
data = "Xenium_5K_OC"
data_dir = f"../../data/{data}/"
output_dir = f"../../output/{data}/"

In [73]:
adata = sc.read_h5ad(data_dir + "intermediate_data/adata.h5ad")
adata

AnnData object with n_obs × n_vars = 327607 × 5001
    obs: 'cell_id', 'global_x', 'global_y', 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'segmentation_method', 'cell_type', 'cell_type_merged'
    var: 'gene_ids', 'feature_types', 'genome', 'gene'
    uns: 'cell_type_colors', 'cell_type_merged_colors'

In [97]:
T = pd.read_csv(data_dir + "raw_data/HE_alignment.csv", header=None).to_numpy(dtype=float)
a = T[0, 0]
b = T[0, 1]
x_shift = T[0, 2]
y_shift = T[1, 2]

# # (n_cells, 2) in micrometers
# XY_um = adata.obs[["global_x", "global_y"]].to_numpy(dtype=float)

# # homogeneous coords: (n_cells, 3)
# XY1 = np.c_[XY_um, np.ones((XY_um.shape[0], 1))]

# # apply transform: (n_cells, 3) = (n_cells, 3) @ (3, 3).T
# UV1 = XY1 @ T.T

# adata.obs["x_pixel"] = UV1[:, 1]
# adata.obs["y_pixel"] = UV1[:, 0]

# Tinv = np.linalg.inv(T)

# XY_um = adata.obs[["global_x", "global_y"]].to_numpy(dtype=float)
# XY1 = np.c_[XY_um, np.ones((XY_um.shape[0], 1))]

# UV1 = XY1 @ Tinv.T   # NOTE: inverse here

# adata.obs["x_pixel"] = UV1[:, 0]
# adata.obs["y_pixel"] = UV1[:, 1]

In [99]:
s = np.sqrt(a*a + b*b)              # pixels per micron
pixel_size = 1.0 / s                # microns per pixel
theta = np.arctan2(b, a)

# microns
x = adata.obs["global_x"].to_numpy(float)
y = adata.obs["global_y"].to_numpy(float)

# rotate in micron space, then scale by 1/pixel_size
x_rot = np.cos(theta) * x + np.sin(theta) * y
y_rot = -np.sin(theta) * x + np.cos(theta) * y

col = (x_rot / pixel_size) + x_shift
row = (y_rot / pixel_size) + y_shift

adata.obs["x_pixel"] = col
adata.obs["y_pixel"] = row

In [105]:
s

1.2895710351401162

In [75]:
# Read image
img_rgb = tifffile.imread(data_dir + "raw_data/Xenium_Prime_Ovarian_Cancer_FFPE_XRrun_he_image.ome.tif")
img_rgb.shape

(43993, 30918, 3)

In [104]:
h, w = img_rgb.shape[:2]

# OpenCV uses BGR
canvas = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)

# -----------------------------
# Overlay cells at full res
# -----------------------------
x = adata.obs["x_pixel"].to_numpy(dtype=float)
y = adata.obs["y_pixel"].to_numpy(dtype=float)

# keep points inside image bounds
mask = (x >= 0) & (x < w) & (y >= 0) & (y < h)
x_in = x[mask].astype(np.int32)
y_in = y[mask].astype(np.int32)

# draw points (tweak radius if too dense)
for xi, yi in zip(x_in, y_in):
    cv2.circle(canvas, (xi, yi), radius=2, color=(0, 0, 255), thickness=-1)

# -----------------------------
# Downsample AFTER plotting
# -----------------------------
shorter_target = 2000
scale = shorter_target / min(h, w)
new_w = int(round(w * scale))
new_h = int(round(h * scale))

canvas_small = cv2.resize(canvas, (new_w, new_h), interpolation=cv2.INTER_AREA)

# -----------------------------
# Save
# -----------------------------
out_path = "HE_with_cells_small.png"
cv2.imwrite(out_path, canvas_small)

True

In [100]:
adata.obs["global_x"].max(), adata.obs["global_x"].min()

(11481.025390625, 12.970833778381348)

In [101]:
adata.obs["global_y"].max(), adata.obs["global_y"].min()

(7976.9521484375, 21.870620727539062)

In [102]:
adata.obs["x_pixel"].max(), adata.obs["x_pixel"].min()

(9647.264894709118, -624.2830123301154)

In [103]:
adata.obs["y_pixel"].max(), adata.obs["y_pixel"].min()

(38689.362557871405, 23872.231979579246)